In [1]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json


In [2]:
# Read price data from Excel
price_data = pd.read_csv('price_data.csv')
price_value = price_data['Price'].values  # Assuming the second column is named 'Price'

# Parameters
Seff = 1
Ceff = 1
Deff = 1
cap_power = 1  # Example value
cap_energy = 4  # Example value
λ = 0.5
η = 1000

initial_soc = λ * cap_energy

# Model

In [3]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    T = range(1, len(price_vector) + 1)
    SOC_T = range(0, len(price_vector) + 1)

    p_t_arb = {t: price_vector[t-1] for t in T}
    start_soc = λ * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals)
    model.C = Var(T, within=NonNegativeReals, bounds = (0, 1))
    model.D = Var(T, within=NonNegativeReals, bounds = (0, 1))
    # model.y_ch = Var(T, within=Binary)
    # model.y_dis = Var(T, within=Binary)

    # Objective function
    model.obj = Objective(
        expr=sum(p_t_arb[t] * cap_power * (model.D[t] * Deff - model.C[t]) for t in T),
        sense=maximize
    )

    # Constraints
    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            return model.SOC[t] == model.SOC[t-1] * Seff + cap_power * (model.C[t] * Ceff - model.D[t])
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    def soc_final_state_constraint(model):
        end_t = SOC_T[-1]
        return model.SOC[end_t] == start_soc #* Seff 
    
    if current_period == last_period:
        model.SOC_final = Constraint(rule=soc_final_state_constraint)

    def soc_cap_constraint(model, t):
        return model.SOC[t] <= cap_energy
    model.SOC_cap = Constraint(SOC_T, rule=soc_cap_constraint)

    def energy_cap_C_constraint(model):
        return sum(model.C[t] for t in T) <= η * cap_energy
    model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D[t] for t in T) <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)

    # def charge_discharge_limit_constraint(model, t):
    #     return model.y_ch[t] + model.y_dis[t] <= 1
    # model.charge_discharge_limit = Constraint(T, rule=charge_discharge_limit_constraint)

    # def c_limit_constraint(model, t):
    #     return model.C[t] <= model.y_ch[t]
    # model.C_limit = Constraint(T, rule=c_limit_constraint)

    # def d_limit_constraint(model, t):
    #     return model.D[t] <= model.y_dis[t]
    # model.D_limit = Constraint(T, rule=d_limit_constraint)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D[t].value for t in T]  # Discharging as positive
    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    # Return the results and final SOC
    return model.obj(), charging_schedule, discharging_schedule, soc_schedule

In [4]:
price = price_value #[:72]

time_period = 24    # periods

num_slices = len(price) // time_period

# Lists to store results
total_revenue = 0
all_charging_schedules = []
all_discharging_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    periodic_price = price[p*time_period:(p+1)*time_period]
    revenue, charging_schedule, discharging_schedule, soc_schedule = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\angelmah\AppData\Local\Temp\tmpog05d5cj.glpk.raw --wglp
 C:\Users\angelmah\AppData\Local\Temp\tmp8yz0q2jz.glpk.glp --cpxlp C:\Users\angelmah\AppData\Local\Temp\tmpn3wkp8p8.pyomo.lp
Reading problem data from 'C:\Users\angelmah\AppData\Local\Temp\tmpn3wkp8p8.pyomo.lp'...
52 rows, 73 columns, 170 non-zeros
456 lines were read
Writing problem data to 'C:\Users\angelmah\AppData\Local\Temp\tmp8yz0q2jz.glpk.glp'...
422 lines were written
GLPK Simplex Optimizer, v4.65
52 rows, 73 columns, 170 non-zeros
Preprocessing...
24 rows, 48 columns, 71 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 24
      0: obj =   2.941400000e+02 inf =   1.000e+00 (1)
      1: obj =   2.816600000e+02 inf =   0.000e+00 (0)
*    37: obj =   5.671800000e+02 inf =   0.000e+00 (0)
OPTIMAL LP S

In [ ]:
# Create a DataFrame for the results
data = {
    't': range(1, len(all_charging_schedules) + 1),
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

In [ ]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['net_discharge'], mode='lines', name='net'))
fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_charge_discharge.show()

# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index, y=result_df['soc']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()

In [ ]:
# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>
        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)